In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [5]:
import PPIGraph

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Read per cell data

In [6]:
from torch_geometric.loader import DataLoader

In [7]:
# Define condition mapping
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}

# Load graph dataset and process if neede
graph_path = data_dir / 'OCT Cell Culture' / 'Whole' / 'graphs' 

In [8]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt',  condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [9]:
dataset

GraphDataset(2064)

In [10]:
dataset_filtered

GraphDataset(1522)

# ML model

In [12]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [14]:
condition = 'ML'
project_name = f'PLA_5PPI_11324_{condition}'

In [15]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True),
    'LogisticRegression':  LogisticRegression(),
    'MLP': MLPClassifier(random_state=1, max_iter=100, hidden_layer_sizes=[16, 16, 16])
}

In [17]:
import torch 

data = torch.empty((0, 5))
label = []

for graph in dataset_filtered:
    new_row =graph.x.sum(axis = 0).unsqueeze(0)
    data = torch.cat((data, new_row), dim=0)
    label.append(graph.condition)


In [18]:
scaler = StandardScaler()

# Run model on cell count
X = data.numpy()
X = scaler.fit_transform(X)
y = np.array(label)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.73115
auc,0.707
b_accuracy,0.707
f1,0.62727


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.71803
auc,0.69267
b_accuracy,0.69267
f1,0.61607


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.72697
auc,0.68527
b_accuracy,0.68527
f1,0.58706


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76316
auc,0.73382
b_accuracy,0.73382
f1,0.66038


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.73355
auc,0.6849
b_accuracy,0.6849
f1,0.58031


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76974
auc,0.74809
b_accuracy,0.74809
f1,0.68182


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.75987
auc,0.72079
b_accuracy,0.72079
f1,0.63317


In [49]:
import torch 

data = torch.empty((0, 10))
label = []

for graph in dataset_filtered:
    cell =graph.x.sum(axis = 0).unsqueeze(0)

    nuclei = torch.tile(graph.nuclei, (5, 1)).T * graph.x
    nuclei = nuclei.sum(axis=0).unsqueeze(0)
    cyto = cell - nuclei
    new_row = torch.cat([cyto, nuclei], axis=1)
    data = torch.cat((data, new_row), dim=0)
    label.append(graph.condition)

In [50]:
scaler = StandardScaler()

# Run model on cell count
X = data.numpy()
X = scaler.fit_transform(X)
y = np.array(label)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cytonuclei', name=model_name+f'_cytonuclei_{k}')

        accuracy = metrics.accuracy_score(y_test, y_pred)
        b_accuracy = metrics.balanced_accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'b_accuracy': b_accuracy, 'f1':f1, 'auc': auc})
    run.finish()

accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.7377
auc,0.71968
b_accuracy,0.71968
f1,0.64602


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76721
auc,0.74533
b_accuracy,0.74533
f1,0.68722


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.69079
auc,0.66034
b_accuracy,0.66034
f1,0.56481


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76316
auc,0.73382
b_accuracy,0.73382
f1,0.66038


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.74342
auc,0.69529
b_accuracy,0.69529
f1,0.59375


...

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.73355
auc,0.69606
b_accuracy,0.69606
f1,0.60488


c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.78618
auc,0.75756
b_accuracy,0.75756
f1,0.69194


accuracy,▁
auc,▁
b_accuracy,▁
f1,▁
accuracy,0.76645
auc,0.72373
b_accuracy,0.72373
f1,0.6359
